In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
df = pd.read_csv('/kaggle/input/aviation-accident-database-synopses/AviationData.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()/len(df)*100

In [ ]:
df['Event.Date'] = pd.to_datetime(df['Event.Date'])
df['Year'] = df['Event.Date'].dt.year
df['Month'] = df['Event.Date'].dt.month_name()
df['Day'] = df['Event.Date'].dt.day_name()

In [ ]:
df['Weather.Condition'] = df['Weather.Condition'].replace('Unk','UNK')
df['Weather.Condition'].fillna('UNK', inplace=True)
df['Weather.Condition'] = df['Weather.Condition'].replace('UNK','Unknown')

In [ ]:
df['Injury.Severity'] = df['Injury.Severity'].str.split('(').str[0]
df['Injury.Severity'] = df['Injury.Severity'].replace('Siruose','Serious')
df['Injury.Severity'] = df['Injury.Severity'].replace('Unavailable','Unknown')

# Number Of Accidents Across Years, Months & Week-Days

In [ ]:
yearly = df.groupby('Year')['Accident.Number'].count().reset_index()
monthly = df.groupby('Month')['Accident.Number'].count().reset_index()
week_day = df.groupby('Day')['Accident.Number'].count().reset_index()

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=False,
                   vertical_spacing=0.15,
                   specs =[[{'type':'scatter'}],
                          [{'type':'bar'}],
                          [{'type':'pie'}]],
                   subplot_titles=['Accidents Over The Years','Month With Most Number Of Accidents','Week Days With Highest Accidents Count'])

trace1 = go.Scatter(x=yearly['Year'],
                         y=yearly['Accident.Number'],
                         mode='lines+markers',
                   line_color='gold')


trace2 = go.Bar(x=monthly['Month'],
                     y=monthly['Accident.Number'],
                     text = monthly['Accident.Number'],
                     marker=dict(color = monthly['Accident.Number'],colorscale='viridis'),
                     textposition='outside')


trace3 = go.Pie(labels=week_day['Day'],
                     values=week_day['Accident.Number'],
                textinfo='percent',
                insidetextorientation='radial',
                legendgroup='show', marker = dict(colors=px.colors.qualitative.Pastel2))

fig.append_trace(trace1,1,1)
fig.append_trace(trace2,2,1)
fig.append_trace(trace3,3,1)

fig.update_traces()
fig.update_layout(uniformtext_minsize=7, uniformtext_mode='show', height=1400, width=1200)
fig.show()

In [ ]:
fig = px.bar(x=df['Investigation.Type'].value_counts().index,
            y=df['Investigation.Type'].value_counts(),
            title='Accident VS Incident',
            text=(df['Investigation.Type'].value_counts()/len(df['Investigation.Type'])*100),
            height=500,
            width=600)

fig.update_traces(textposition='outside', texttemplate='%{text:.4s}%', marker=dict(color='snow', line=dict(color='black', width=3)))
fig.show()

**Incident Fatalities  Analysis**

In [ ]:
incident = df[df['Investigation.Type']=='Incident']
accident = df[df['Investigation.Type']=='Accident']


incident_injuries = incident.groupby('Year')[['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']].sum().reset_index()

accident_injuries = incident.groupby('Year')[['Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']].sum().reset_index()

In [ ]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=False,
                   vertical_spacing=0.03,
                   specs =[[{'type':'table'}],
                          [{'type':'scatter'}],
                          [{'type':'scatter'}],
                          [{'type':'scatter'}],
                          [{'type':'scatter'}]],
                   subplot_titles=['','Fatal Injuries In Incident Over The Years','Serious Injuries In Incident Over The Year',
                                  'Minor Injuries In Incident Over The Years','UnInjured In Incident Over The Year'])

fig.add_trace(
    go.Scatter(
        x=incident_injuries["Year"],
        y=incident_injuries["Total.Fatal.Injuries"],
        mode="lines+markers",
        name="Fatal Injuries",
        line_color='black'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
    x=incident_injuries['Year'],
    y=incident_injuries['Total.Serious.Injuries'],
    mode='lines+markers',
    name='Serious Injuries',
    line_color='purple'),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=incident_injuries["Year"],
        y=incident_injuries["Total.Minor.Injuries"],
        mode="lines+markers",
        name="Minor Injuries",
        line_color='grey'
    ),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(
        x=incident_injuries["Year"],
        y=incident_injuries["Total.Uninjured"],
        mode="lines+markers",
        name="Uninjured",
        line_color='navy'
    ),
    row=5, col=1
)


fig.add_trace(
    go.Table(
        header=dict(
            values=["Year", 'Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured'],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[incident_injuries['Year'],incident_injuries['Total.Fatal.Injuries'],incident_injuries['Total.Serious.Injuries'],
                    incident_injuries['Total.Minor.Injuries'],incident_injuries['Total.Uninjured']],align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=1500,
    showlegend=False,
    title_text="Incident Injuries Analysis",
)

fig.show()

**Accidents Fatalities Analysis**

In [ ]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=False,
                   vertical_spacing=0.03,
                   specs =[[{'type':'table'}],
                          [{'type':'scatter'}],
                          [{'type':'scatter'}],
                          [{'type':'scatter'}],
                          [{'type':'scatter'}]],
                    subplot_titles=['','Fatal Injuries In Accident Over The Years','Serious Injuries In Accident Over The Year',
                                  'Minor Injuries In Accident Over The Years','UnInjured In Accident Over The Year'])

fig.add_trace(
    go.Scatter(
        x=accident_injuries["Year"],
        y=accident_injuries["Total.Fatal.Injuries"],
        mode="lines+markers",
        name="Fatal Injuries",
        line_color='brown'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
    x=accident_injuries['Year'],
    y=accident_injuries['Total.Serious.Injuries'],
    mode='lines+markers',
    name='Serious Injuries',
    line_color='cadetblue'),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=accident_injuries["Year"],
        y=accident_injuries["Total.Minor.Injuries"],
        mode="lines+markers",
        name="Minor Injuries",
        line_color='olive'
    ),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(
        x=accident_injuries["Year"],
        y=accident_injuries["Total.Uninjured"],
        mode="lines+markers",
        name="Uninjured",
        line_color='darkorange'
    ),
    row=5, col=1
)


fig.add_trace(
    go.Table(
        header=dict(
            values=["Year", 'Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured'],
            font=dict(size=10),
            align="left"
        ),cells=dict(
            values=[accident_injuries['Year'],incident_injuries['Total.Fatal.Injuries'],incident_injuries['Total.Serious.Injuries'],
                    accident_injuries['Total.Minor.Injuries'],incident_injuries['Total.Uninjured']],align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=1500,

    showlegend=False,
    title_text="Accident Injuries Analysis",
)

fig.show()

***Analysing Purpose Of Flight With Accidents Counts are Greater Than 10*** 
 
 ***Purpose Of Flights With Their Fatalities Rate & Percentage***

In [ ]:
print(df['Purpose.of.flight'].nunique())
print('')
print(df['Purpose.of.flight'].unique())

In [ ]:
total_fatal = df[['Purpose.of.flight','Total.Fatal.Injuries']].groupby('Purpose.of.flight').agg(['sum','count'])
total_fatal = total_fatal['Total.Fatal.Injuries'].reset_index()
total_fatal['fatal_percentage'] = total_fatal['sum']/total_fatal['count']
total_fatal.columns = ['Purpose.of.flight','Total.Fatal.Injuries','Total_Accidents','Fatal_Percentage']



total_uninjured =  df[['Purpose.of.flight','Total.Uninjured']].groupby('Purpose.of.flight').agg(['sum','count'])
total_uninjured = total_uninjured['Total.Uninjured'].reset_index()
total_uninjured['Uninjured_percentage'] = total_uninjured['sum']/total_uninjured['count']
total_uninjured.columns = ['Purpose.of.flight','Total.Uninjured','Total_Accidents','Uninjured_Percentage']

In [ ]:
minacc = 10
propstoplot = total_fatal[total_fatal['Total_Accidents']>minacc]
propstoplot = propstoplot.sort_values('Fatal_Percentage').tail(50)


fig = go.Figure()
fig.add_trace(go.Bar(
   x=propstoplot['Purpose.of.flight'],
   y=propstoplot['Total_Accidents'],
   text=propstoplot['Total_Accidents'],
   name='Purpose Of Flight Accident > 10 Accidents',
   marker_color='grey'
))

fig.add_trace(go.Bar(
    x=propstoplot['Purpose.of.flight'],
    y=propstoplot['Total.Fatal.Injuries'],
    text=propstoplot['Total.Fatal.Injuries'],
    name='Fatal Injury Count',
    marker_color = 'silver'
))

fig.update_traces(textposition='outside', texttemplate='%{text:.2s}')
fig.update_layout(barmode='group', xaxis_tickangle=-45, height=600, width=1300,uniformtext_minsize=7, uniformtext_mode='show',
                 title='Purpose Of Flight Total Accidents VS Total Fatal Injuries ')
fig.show()



fig = px.bar(x=propstoplot['Purpose.of.flight'],
   y=propstoplot['Fatal_Percentage'],
   text=propstoplot['Fatal_Percentage'],
   title='Purpose Of Flight Fatal Percentage')


fig.update_traces(textposition='outside', texttemplate='%{text:.4s}', marker = dict(color='antiquewhite', line=dict(color='peru', width=3)))
fig.update_layout( height=600, width=1100,uniformtext_minsize=8, uniformtext_mode='show')
fig.show()


**Findings:**

1. Although Skydiving Accident Counts are low but their Death rate is high.

2. Count of Accidents happens to flight taken for personal purposes are highest.

3. Aerial Application flights has the lowest Death percentage.

***Analysing Purpose Of Flight With Accidents Counts are Greater Than 10***


**Purpose Of Flight With Their Uninjured Rate & Percentage**

In [ ]:



uninjured = total_uninjured[total_uninjured['Total_Accidents']>minacc]
uninjured = uninjured.sort_values('Uninjured_Percentage').tail(50)


# Plot For Total Accidents and Total Uninjured For purpose of Flight
fig = go.Figure()
fig.add_trace(go.Bar(
   x=uninjured['Purpose.of.flight'],
   y=uninjured['Total_Accidents'],
   text=uninjured['Total_Accidents'],
   name='Purpose Of Flight Accident > 10 Accidents',
   marker_color='yellowgreen'
))

fig.add_trace(go.Bar(
    x=uninjured['Purpose.of.flight'],
    y=uninjured['Total.Uninjured'],
    text=uninjured['Total.Uninjured'],
    name='Uninjured Count',
    marker_color = 'olive'
))

fig.update_traces(textposition='outside', texttemplate='%{text:.2s}')
fig.update_layout(barmode='group', xaxis_tickangle=-45, height=600, width=1300,uniformtext_minsize=7, uniformtext_mode='show',
                 title='Purpose Of Flight Total Accidents VS Total UnInjured')
fig.show()



fig = px.bar(x=uninjured['Purpose.of.flight'],
    y=uninjured['Uninjured_Percentage'],
    text=uninjured['Uninjured_Percentage'],
    title='Purpose Of Flight Uninjured Percentage')

fig.update_traces(textposition='outside', texttemplate='%{text:.3s}', marker = dict(color='mintcream', line=dict(color='teal', width=3)))
fig.update_layout( height=600, width=1100,uniformtext_minsize=8, uniformtext_mode='show')
fig.show()




 ***Purpose Of Flight With Injury Severity.***

In [ ]:
# Analysing flights whose value_counts are greater than 100

flight_injury = df[df['Purpose.of.flight'].map(df['Purpose.of.flight'].value_counts()) >=100]

flight_injury = flight_injury.groupby('Purpose.of.flight')['Injury.Severity'].value_counts().reset_index(name='count')

fig = px.histogram(x=flight_injury['Purpose.of.flight'],
                  y=flight_injury['count'],
                  color=flight_injury['Injury.Severity'],
                  barmode='group',
                  color_discrete_sequence=['hotpink','plum','coral','cadetblue','lightseagreen','silver'],
                  title='Purpose Of Flight & Injury Severity')
fig.show()

In [ ]:
skydiving = df[df['Purpose.of.flight']=='Skydiving']
personal = df[df['Purpose.of.flight']=='Personal']
business = df[df['Purpose.of.flight']=='Business']

personal_y = personal.groupby('Year')['Accident.Number'].count().reset_index(name='count')
personal_m = personal.groupby('Month')['Accident.Number'].count().reset_index(name='count').sort_values(by='count',ascending=False)
personal_d = personal.groupby('Day')['Accident.Number'].count().reset_index(name='count').sort_values(by='count',ascending=False)

business_y = business.groupby('Year')['Accident.Number'].count().reset_index(name='count')
business_m = business.groupby('Month')['Accident.Number'].count().reset_index(name='count').sort_values(by='count',ascending=False)
business_d = business.groupby('Day')['Accident.Number'].count().reset_index(name='count').sort_values(by='count',ascending=False)

skydiving_y = skydiving.groupby('Year')['Accident.Number'].count().reset_index(name='count')
skydiving_m = skydiving.groupby('Month')['Accident.Number'].count().reset_index(name='count').sort_values(by='count',ascending=False)
skydiving_d = skydiving.groupby('Day')['Accident.Number'].count().reset_index(name='count').sort_values(by='count',ascending=False)

***Analysing Flights Taken For Personal Reasons ***

In [ ]:
fig = make_subplots(rows=2, cols=2,
                   specs=[[{'type':'bar'},{'type':'pie'}],
                         [{'type':'scatter'},None]],
                   subplot_titles=['Personal Purpose Flight Accidents Across The Months',
                                  'Personal Purpose Flight Accidents Across The Week-Days',
                                  'Personal Purpose Flight Accidents Across The Year'])

trace1 = go.Bar(x=personal_m['Month'],
                y=personal_m['count'],
               text=personal_m['count'],
               marker=dict(color = personal_m['count'],colorscale='mint'),
               textposition='outside')


trace2 = go.Pie(labels=personal_d['Day'],
                values=personal_d['count'],
                textinfo='percent',
                insidetextorientation='radial',
                legendgroup='show', 
                marker = dict(colors=px.colors.qualitative.Pastel))


trace3 = go.Scatter(x=personal_y['Year'],
                   y=personal_y['count'],
                   mode='lines+markers',
                    line_color='red',
                   fill='toself'
                   )

fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace3,2,1)

fig.update_layout(height=800, width=1400)

fig.show()

**Findings:**

1. Most people goes to vacation during summer holidays thats why count of accidents happens to flights taken for personal reasons are high in the months of July, june and August.

2. Accident rate for personal reasons are high during the weekends too.

3. Data seems to be incomplete as the count of accidents for personal reason flight is 1 from year 1948 to 1981.

***Analysing Flights Taken For Business Purposes***

In [ ]:
fig = make_subplots(rows=2, cols=2,
                   specs=[[{'type':'bar'},{'type':'pie'}],
                         [{'type':'scatter'},None]],
                   subplot_titles=['Business Purpose Flight Accidents Across The Months',
                                  'Business Purpose Flight Accidents Across The Week-Days',
                                  'Business Purpose Flight Accidents Across The Years'])

trace1 = go.Bar(x=business_m['Month'],
                y=business_m['count'],
               text=business_m['count'],
               marker=dict(color = business_m['count'],colorscale='purples'),
               textposition='outside')


trace2 = go.Pie(labels=business_d['Day'],
                values=business_d['count'],
                textinfo='percent',
                insidetextorientation='radial',
                legendgroup='show', 
                marker = dict(colors=px.colors.qualitative.Set3))


trace3 = go.Scatter(x=business_y['Year'],
                   y=business_y['count'],
                   mode='lines+markers',
                    line_color='darkkhaki',
                   fill='tonexty')

fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace3,2,1)

fig.update_layout(height=900, width=1500)

fig.show()

**Findings:**

1. Month of August has the highest accidents count for flight for business purposes.

2. Most of business trip happens during week days that why Accidents count during the weed days are high compared to weekends.


***Analysing Skydivings***

In [ ]:
fig = make_subplots(rows=2, cols=2,
                   specs=[[{'type':'bar'},{'type':'pie'}],
                         [{'type':'scatter'},None]],
                   subplot_titles=['Skydiving Accidents Across The Months','Skydiving Accidents Across The Week-Days',
                                 'Skydiving Accidents Across The Years'])

trace1 = go.Bar(x=skydiving_m['Month'],
                y=skydiving_m['count'],
               text=skydiving_m['count'],
               marker=dict(color = skydiving_m['count'],colorscale='earth'),
               textposition='outside')


trace2 = go.Pie(labels=skydiving_d['Day'],
                values=skydiving_d['count'],
                textinfo='percent',
                insidetextorientation='radial',
                legendgroup='show', 
                marker = dict(colors=px.colors.qualitative.Set2))


trace3 = go.Scatter(x=skydiving_y['Year'],
                   y=skydiving_y['count'],
                   mode='lines+markers',
                    line_color='lightsalmon',
                   fill='tonexty')

fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig.append_trace(trace3,2,1)

fig.update_layout(height=900, width=1500)

fig.show()

**Findings:**

1. Skydiving Accidents rate is alot higher on weekends compare to week-days.

2. High count of skydiving accidents happens during the summer holiday months.

In [ ]:
print(df['Air.carrier'].nunique())
print('')
print(df['Air.carrier'].value_counts().head(15))

***Which Air Carrier Has The Most Accident Occurance?***

In [ ]:
air_carrier = df[['Air.carrier','Total.Fatal.Injuries']].groupby(by='Air.carrier').agg(['sum','count'])
air_carrier = air_carrier['Total.Fatal.Injuries'].reset_index()
air_carrier['Carrier_Fatalities_Per'] = air_carrier['sum']/air_carrier['count']
air_carrier.columns =['Air.carrier','Carrier_Injuries','Carrier_Accidents','Carrier_Fatalities_Per']


carrier_uninjured =  df[['Air.carrier','Total.Uninjured']].groupby('Air.carrier').agg(['sum','count'])
carrier_uninjured = carrier_uninjured['Total.Uninjured'].reset_index()
carrier_uninjured['Uninjured_percentage'] = carrier_uninjured['sum']/carrier_uninjured['count']
carrier_uninjured.columns = ['Air.carrier','Total.Uninjured','Total_Accidents','Uninjured_Percentage']

***Analysing Air Carrier Accidents With Counts Greater than 10***

***Air Carrier Accidents With Fatal Injury Analysis***

In [ ]:
carrier = air_carrier[air_carrier['Carrier_Accidents']>minacc]
carrier = carrier.sort_values('Carrier_Fatalities_Per').tail(50)

uninjured = carrier_uninjured[carrier_uninjured['Total_Accidents']>minacc]
uninjured = uninjured.sort_values('Uninjured_Percentage').tail(50)

fig = go.Figure()


fig.add_trace(go.Bar(
   x=carrier['Air.carrier'],
   y=carrier['Carrier_Accidents'],
   text=carrier['Carrier_Accidents'],
   name='Total Air Carrier Accidents',
   marker_color='steelblue'
))

fig.add_trace(go.Bar(
    x=carrier['Air.carrier'],
    y=carrier['Carrier_Injuries'],
    text=carrier['Carrier_Injuries'],
    name='Fatal Injuries Count Of Air Carrier',
    marker_color = 'powderblue'
))

fig.update_traces(textposition='outside', texttemplate='%{text:.2s}')
fig.update_layout(barmode='group', xaxis_tickangle=-45, height=600, width=1300,uniformtext_minsize=7, uniformtext_mode='hide',
                 title='Total AirCarrier Accidents & Fatal Injuries Count')
fig.show()




fig = px.bar(x=carrier['Air.carrier'],
   y=carrier['Carrier_Fatalities_Per'],
   text=carrier['Carrier_Fatalities_Per'],
   title='Air Carrier Fatal Percentage',)

fig.update_traces(textposition='outside', texttemplate='%{text:.2s}', marker=dict(color='beige', line=dict(color='darkgrey', width=3)))
fig.update_layout( height=600, width=1000,uniformtext_minsize=7, uniformtext_mode='hide')
fig.show()

**Findings:**

1. United Airlines has the highest accidents and fatalities rate.


***Air Carrier Accidents Without Injury Analysis***

In [ ]:

fig = go.Figure()

fig.add_trace(go.Bar(
   x=uninjured['Air.carrier'],
   y=uninjured['Total_Accidents'],
   text=uninjured['Total_Accidents'],
   name='Total Air Carrier Accidents',
   marker_color='hotpink'
))

fig.add_trace(go.Bar(
    x=uninjured['Air.carrier'],
    y=uninjured['Total.Uninjured'],
    text=uninjured['Total.Uninjured'],
    name='Uninjured Count Of Air Carrier',
    marker_color = 'lightpink'
))

fig.update_traces(textposition='outside', texttemplate='%{text:.2s}')
fig.update_layout(barmode='group', xaxis_tickangle=-45, height=600, width=1300,uniformtext_minsize=7, uniformtext_mode='hide',
                 title='Total Air Carrier Accidents & Uninjured Count')
fig.show()




fig = px.bar(x=uninjured['Air.carrier'],
    y=uninjured['Uninjured_Percentage'],
    text=uninjured['Uninjured_Percentage'],
    title='Air Carrier Uninjured Percentage')


fig.update_traces(textposition='outside', texttemplate='%{text:.4s}',marker=dict(color='silver', line=dict(color='black', width=3)))
fig.update_layout(height=600, width=1000,uniformtext_minsize=7, uniformtext_mode='hide')
fig.show()


***AirCraft Damage Analysis***

In [ ]:
df1 = df[df['Aircraft.damage'] !='Unknown']

damage = df1[['Aircraft.damage','Total.Fatal.Injuries']].groupby('Aircraft.damage').agg(['sum','count'])
damage = damage['Total.Fatal.Injuries'].reset_index()
damage['Fatalities_Per'] = damage['sum']/damage['count']
damage.columns = ['Aircraft.damage','Total_Fatalities','Total_Accidents','Fatalities_Per']


uninjured_damage = df1[['Aircraft.damage','Total.Uninjured']].groupby('Aircraft.damage').agg(['sum','count'])
uninjured_damage = uninjured_damage['Total.Uninjured'].reset_index()
uninjured_damage['Uninjured_Per'] = uninjured_damage['sum']/uninjured_damage['count']
uninjured_damage.columns = ['Aircraft.damage','Total_Uninjured','Total_Accidents','Uninjured_Per']


In [ ]:
fig = go.Figure()


fig.add_trace(go.Bar(
   x=damage['Aircraft.damage'],
   y=damage['Total_Accidents'],
   text=damage['Total_Accidents'],
   name='Total Air Damage Accidents',
   marker_color='steelblue'
))

fig.add_trace(go.Bar(
    x=damage['Aircraft.damage'],
    y=damage['Total_Fatalities'],
    text=damage['Total_Fatalities'],
    name='Fatal Injuries Count Of Air Damage',
    marker_color = 'powderblue'
))

fig.update_traces(textposition='outside', texttemplate='%{text:.2}')
fig.update_layout(barmode='group', xaxis_tickangle=-45, height=600, width=1000,uniformtext_minsize=7, uniformtext_mode='hide',
                 title='Air Carrier Fatally Injured Count')
fig.show()





fig = px.bar(x=damage['Aircraft.damage'],
   y=damage['Fatalities_Per'],
   text=damage['Fatalities_Per'],
   title='Air Damage Fatal Percentage',
            height=600,
            width=600)


fig.update_traces(textposition='outside', texttemplate='%{text:.3}', marker=dict(color='mistyrose', line=dict(color='crimson', width=3)))
fig.show()

In [ ]:
fig = go.Figure()


fig.add_trace(go.Bar(
   x=uninjured_damage['Aircraft.damage'],
   y=uninjured_damage['Total_Accidents'],
   text=uninjured_damage['Total_Accidents'],
   name='Total Air Damage Accidents',
   marker_color='plum'
))

fig.add_trace(go.Bar(
    x=uninjured_damage['Aircraft.damage'],
    y=uninjured_damage['Total_Uninjured'],
    text=uninjured_damage['Total_Uninjured'],
    name='Uninjured Count Of Air Damage',
    marker_color = 'mediumorchid'
))

fig.update_traces(textposition='outside', texttemplate='%{text:.2}')
fig.update_layout(barmode='group', xaxis_tickangle=-45, height=600, width=1000,uniformtext_minsize=7, uniformtext_mode='hide',
                 title='Air Carrier UnInjured Count')
fig.show()



fig = px.bar(x=uninjured_damage['Aircraft.damage'],
    y=uninjured_damage['Uninjured_Per'],
    text=uninjured_damage['Uninjured_Per'],
    title='Air Damage Uninjured Percentage',
    height=500,
    width=600)


fig.update_traces(textposition='outside', texttemplate='%{text:.3s}',marker=dict(color='ivory', line=dict(color='darkorange', width=3)))
fig.show()



**Amateur Built Analysis**

In [ ]:
fig = px.bar(x=df['Amateur.Built'].value_counts().index,
            y=df['Amateur.Built'].value_counts(),
            text=(df['Amateur.Built'].value_counts()/len(df['Amateur.Built'])*100),
            title='Amateur Built',
            height=500,
            width=500
            )
fig.update_traces(textposition='outside', texttemplate='%{text:.4s}%', 
                  marker=dict(color='azure', line=dict(color='black', width=3)))
fig.show()

**Almost 90% of aircrafts mentioned in the data are built by professionals**

In [ ]:
aircraft_built = df.groupby('Aircraft.damage')['Amateur.Built'].value_counts().reset_index(name='count')
aircraft_built = aircraft_built[aircraft_built['Amateur.Built']!='Unknown']
aircraft_built.style.background_gradient(cmap='BuGn')

In [ ]:
fig= px.bar(x=aircraft_built['Aircraft.damage'],
           y=aircraft_built['count'],
           color=aircraft_built['Amateur.Built'],
           text=aircraft_built['count'],
           barmode='relative',
           height=500,
           width=600,
           color_discrete_sequence=['darkmagenta','plum'],
           title='Aircraft Damage & Amateur Built')
fig.show()

**Amateur Built Fatal VS Survived Ratio**

In [ ]:
severity = df[(df['Injury.Severity']=='Fatal') | (df['Injury.Severity']=='Non-Fatal') | (df['Injury.Severity']=='Minor') | (df['Injury.Severity']=='Serious')]
severity = severity.groupby('Amateur.Built')['Injury.Severity'].value_counts().reset_index(name='count')
severity.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(x=severity['Amateur.Built'],
            y=severity['count'],
            color=severity['Injury.Severity'],
            barmode='group',
            height=500,
            width=700,
            title='Amateur Built & Injury Severity')
fig.update_traces(marker=dict(color= ['beige','bisque'], line=dict(color='black', width=3)))
fig.show()

**Location Analysis**

In [ ]:
print('There are ',df['LOCATION'].nunique(), ' number unique Locations mention in data')
print('')
df['LOCATION'].value_counts().head(20)

**ANCHORAGE** airport has the highest accident count.

**Phases Of Flight Analysis**

In [ ]:
fig = px.bar(x=df['Broad.phase.of.flight'].value_counts().index,
            y=df['Broad.phase.of.flight'].value_counts(),
            text=(df['Broad.phase.of.flight'].value_counts()/len(df['Broad.phase.of.flight'])*100),
            height=500,
            width=900,
            title='Phases Of Flight')

fig.update_traces(textposition='outside',texttemplate='%{text:.4s}%', marker=dict(color='lavender', line=dict(color='steelblue', width=3)))
fig.show()

**Location & Purpose Of Flight**

****Analysis of locations with value counts >= 150 ****

In [ ]:
location = df[df['LOCATION'].map(df['LOCATION'].value_counts()) >= 150]
locations = location.groupby('LOCATION')['Purpose.of.flight'].value_counts().reset_index(name='count').sort_values(by='count', ascending=False)
locations.style.background_gradient(cmap='BuPu')

In [ ]:
fig = px.histogram(x=locations['LOCATION'],
                  y=locations['count'],
                  color=locations['Purpose.of.flight'],
                  barmode='group',
                  height=500,
                  template='gridon',
                  title='Location & Purpose Of Flight')
fig.show()

In [ ]:
location_phase = location.groupby('LOCATION')['Broad.phase.of.flight'].value_counts().reset_index(name='count').sort_values(by='count', ascending=False)
location_phase.style.background_gradient(cmap='viridis')

In [ ]:
fig = px.histogram(x=location_phase['LOCATION'],
                  y=location_phase['count'],
                  color=location_phase['Broad.phase.of.flight'],
                  barmode='group',
                  template='xgridoff',
                  title='Location & Phases Of Flight')
fig.show()